In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import pandas as pd
from tqdm import tqdm

In [2]:
load_lst = pd.read_excel("거리추가.xlsx")

In [ ]:
url = 'https://zeropay.or.kr/UI_HP_009_03.act'
options = webdriver.ChromeOptions()
driver = webdriver.Chrome(options=options)

driver.get(url)

# 도로명 입력 버튼 클릭
search = driver.find_element(By.CSS_SELECTOR, 'div.storeFind__title > div > label > span')
search.click()

for num in tqdm(range(len(load_lst))):

    name = ''
    add = ''
    temp_name = load_lst.loc[num,'가맹점명']
    temp_add = load_lst.loc[num,'주소']

    # 이름처리
    temp = temp_name.split()
    
    if temp[-1].endswith("점"):  # 매장명에 '점'이 포함되어 있는 경우
        
        if len(temp) == 1:  # 매장명에 띄어쓰기가 없는 경우
            
            if len(temp[0]) > 4:  # 매장명이 4글자 초과인 경우
                name = temp[0][:-3]  # 뒤에 지점명이 포함되어있을지 모르는 3글자는 제외
            else:  # 매장명이 4글자 이하인 경우
                name = temp[0]  # 매장명 그대로 사용
        else:
            temp.remove(temp[-1])  # 마지막 요소(지점명) 제거
            name = "".join(temp)
           
    else:  # 매장명에 '점'이 포함되어 있지 않는 경우 
        name = temp_name  # 매장명 그대로 사용
    
    # 주소처리
    add = temp_add.split()[1]+" "+temp_add.split()[2]

    # 매장명 입력
    search = driver.find_element(By.CSS_SELECTOR, '#iptText')
    search.clear()
    search.send_keys(name)

    # 도로명 입력
    search = driver.find_element(By.CSS_SELECTOR, '#iptText2')
    search.clear()
    search.send_keys(add)


    key = 0
    while key == 0:

        try:
            search = driver.find_element(By.CSS_SELECTOR, '#btnSearch')
            search.click()
            WebDriverWait(driver, 10).until(EC.invisibility_of_element_located((By.CSS_SELECTOR, "[class$='loading__bar']")))
            key=1
        except:
            pass

    search = driver.find_element(By.CSS_SELECTOR, '#tbTbody > tr > td')

    if '없습니다' in search.text:
        load_lst.loc[num,'제로페이'] = "X"
    else:
        load_lst.loc[num,'제로페이'] = "O"
        load_lst.loc[num,'제로페이분류'] = search.text

In [10]:
load_lst

,가맹점명,주소,영업시간,분류,점수,건수,거리,제로페이,제로페이분류
0,라메흐꺄도,서울 영등포구 선유로17길 24 신일아르디세 1층 105~106호,영업시간 매일 11:00 ~ 21:20 · 휴게시간 매일 15:30 ~ 16:30,"해물,생선",4.1,43.0,342.84,O,한식 일반 음식점업
1,타이산,서울 영등포구 선유로13길 25 에이스하이테크시티2 지식산업센터 116호,NaN,중국요리,3.8,9.0,136.22,O,중식 음식점업
2,화사랑화로구이 문래점,"서울 영등포구 선유로9가길 16 광양림빌딩 1,2층",영업시간 매일 11:30 ~ 22:00,"육류,고기",3.5,27.0,300.36,O,한식 일반 음식점업
3,화덕으로간고등어반상,서울 영등포구 선유로9길 10 문래SKV1센터 1층 119호,영업시간 월~토 11:00 ~ 21:00 · 휴게시간 월~토 15:00 ~ 17:00,"해물,생선",4.5,2.0,188.95,X,한식 일반 음식점업
4,쁘띠문래,서울 영등포구 선유로17길 24 신일아르디세 1층 109호,영업시간 화~토 12:00 ~ 22:00 · 휴게시간 화~토 15:00 ~ 17:00,양식,4.1,23.0,342.84,O,한식 해산물 요리 전문점
...,...,...,...,...,...,...,...,...,...
414,더문,서울 영등포구 선유로 27,NaN,술집,0.0,0.0,338.31,X,NaN
415,그린스푼,서울 영등포구 선유로3길 10 하우스디비즈 지하1층 B101호,NaN,한식,0.0,0.0,265.02,X,NaN
416,먹태랑노가리,서울 영등포구 도림로 461,NaN,"호프,요리주점",0.0,0.0,711.60,X,NaN
417,골든푸드코리아,서울 영등포구 도림로 456 상가 101동 105호,NaN,한식,0.0,0.0,777.66,O,한식 일반 음식점업


In [11]:
load_lst.to_excel('제로페이.xlsx')